# An Analysis of Paleo to present Climate Change Data

<details>

<summary> Student Details </summary>

| Student| Details |
| -------- | -------- |
| Course: | KDATG_L08_Y1 |
| Author: | Rebecca Hannah Quinn |
| Student Number: | G00425671 |

</details>

---

## Project Goals

To Analyse CO2 vs Temperature Anomaly for the past 800,000 years to the present day.
To examine the change in the polar-ice coverage alongside this.
To examine the changes in Ireland and Irish climate change signals.

In gathering this information and data I will fuse and analyse the data using pandas dataframes and export the results to csv and json formats.

I will be anaylising the data, the trends, the relationships between them including but not limited to: temporal leads, lags, and frequencies. I will also use synthesised data to predict global temperature anomoly over the next few decades and compare to published models of the same to see if atmospherice CO2 trends continue and comment on the accelerated warming based on the latest features with temperatures polar-ice coverage and ocean and sea levels.




<details>

<summary> Table of Contents </summary>

[INTRODUCTION](#01)

[PRE-PROCESSING](#02)

[ANALYSIS](#03)

[PREDICITONS](#04)

[FURTHER COMMENT](#05)

[PLOTS](#06)

</details>

---


## Introduction


There have been studies on the Earth"s climate that have resulted in data as far back as 800,000 years by extracting core samples from deep beneath the ice sheets of Greenland and Antarctica. These samples contain detailed information on air temperature and CO2 levels that are trapped within them. Current polar records demonstrate a close association between atmospheric carbon dioxide and temperature in the natural world. In essence, when one increases, the other one follows.

However, there is still some uncertainty about which occurred first - a spike in temperature or CO2. Until now, the most extensive records to date on a significant change in Earth"s climate came from the EPICA Dome C ice core on the Antarctic Plateau. The data, which covered the end of the last ice age, between 20,000 and 10,000 years ago, indicated that CO2 levels could have lagged behind rising global temperatures by as much as 1,400 years.
[1]: https://www.scientificamerican.com/article/ice-core-data-help-solve/#:~:text=Scientists%20use%20air%20trapped%20in,than%20the%20ice%20surrounding%20them.


<a id="01i">

## Pre-processing

</a>





## Import Packages

In [ ]:
#importing packages required for analysis and visualization
import requests
from io import StringIO
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Reading in the Data

### CO2

#### CO2 Data 1 - NCEI 1855 - 1960

In [ ]:
epica1col = ["year", "co2", "sigma"]
epica8kyr1 = pd.read_excel("https://www.ncei.noaa.gov/pub/data/paleo/icecore/antarctica/antarctica2015co2.xls", sheet_name="CO2 Composite", skiprows=59, names=epica1col)
epica8kyr1["year"] = epica8kyr1["year"].astype(int)
epica8kyr1["year"] = -1 * epica8kyr1["year"] + 1950
epica8kyr1 = epica8kyr1.sort_values(by="year", ascending=False)
epica8kyr1.to_csv("epica8kyr1.csv", index=False)

#### CO2 Data 2 - Global Monitoring Laboratory - 1959 - 2023

In [ ]:
epica2col = ["year", "co2", "unc"]
epica8kyr2 = pd.read_csv("https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_annmean_mlo.csv", skiprows=44, sep=",", names=epica2col)

#### CO2 Data 3 - Mauna Lon Observeratory - 1984 - 2023

In [ ]:
epica3col = ["year", "co2", "unc"]
url3 = "https://gml.noaa.gov/webdata/ccgg/trends/ch4/ch4_annmean_gl.txt"
response = requests.get(url3)
if response.status_code == 200:
    text = response.text
    epica8kyr3 = pd.read_fwf(StringIO(text), names=epica3col, skiprows=45, header=0)
    epica8kyr3 = epica8kyr3.sort_values(by="year", ascending=False)  #corrected
    epica8kyr3.to_csv("epica8kyr3.csv", index=False)

##### Initial Analysis

###### Epica CO2 Data 1

In [ ]:
epica8kyr1.head() #head function to check first data lines

In [ ]:
epica8kyr1.tail()

In [ ]:
epica8kyr1.columns #info and columns for merging

In [ ]:
epica8kyr1.info()

---

###### Epica CO2 Data 2

In [ ]:
epica8kyr2.head()

In [ ]:
epica8kyr2.tail()

In [ ]:
epica8kyr2.columns

In [ ]:
epica8kyr2.info()

---

###### Epica CO2 Data 3

In [ ]:
epica8kyr3.head()

In [ ]:
epica8kyr3.tail()

In [ ]:
epica8kyr3.columns

In [ ]:
epica8kyr3.info()

---

---

### CH4

#### CH4 Data 1

In [ ]:
colnames = ["year", "ch4", "unc"]
noaach4 = "https://gml.noaa.gov/webdata/ccgg/trends/ch4/ch4_annmean_gl.txt"

response = requests.get(noaach4)

if response.status_code == 200:
    text = response.text
    noaach4df = pd.read_fwf(StringIO(response.text), names=colnames, widths=[6, 12, 8], skiprows=44, header=0)
    noaach4sort = noaach4df.sort_index(ascending=False)
    noaach4sort.to_csv("noaach4new.csv", index=False)

#### CH4 Data 2

In [ ]:
ch4names = ["year", "ch4"]
ch4data = pd.read_csv("https://climatechange.chicago.gov/sites/production/files/2016-08/ghg-concentrations_fig-2.csv", usecols=[0, 1], skiprows=7, header=0, names=ch4names)
ch4data["year"] = pd.to_numeric(ch4data["year"], errors='coerce')
ch4data = ch4data.dropna(subset=["year"])
ch4data["year"] = ch4data["year"].astype(int)
ch4data.to_csv("ch4data.csv")

##### Initial Analysis

##### CH4 DATA 1

In [ ]:
noaach4sort.head()

In [ ]:
noaach4sort.tail()

In [ ]:
noaach4sort.info()

In [ ]:
noaach4sort.describe()

In [ ]:
noaach4sort.columns

##### CH4 DATA 2

In [ ]:
ch4data.head()

In [ ]:
ch4data.tail()

In [ ]:
ch4data.info()

In [ ]:
ch4data.describe()

In [ ]:
ch4data.columns

---

---

### Temperature

#### Temperature Data 1

In [ ]:
epicadeut = "https://www.ncei.noaa.gov/pub/data/paleo/icecore/antarctica/epica_domec/edc3deuttemp2007.txt"

response = requests.get(epicadeut)

if response.status_code == 200:
    text = response.text
    epicadeutdf = pd.read_csv(StringIO(text), sep='\s+', skiprows=89, header=0)
    epicadeutdf.rename(columns={"Age": "year"}, inplace=True)
    epicadeutdf.rename(columns={"Temperature": "temp"}, inplace=True)
    epicadeutdf["year"] = epicadeutdf["year"].astype(int)
    epicadeutdf["year"] = -1 * epicadeutdf["year"] + 1950
    epicadeutdf["temp"] = epicadeutdf["temp"] + (-54.5)
    
    epicacols = epicadeutdf.iloc[:, [2, 4]]

epicacols.to_csv("epicadeut.csv", index=False)

#### Temperature Data 2

In [ ]:
noaanames=["year", "temp"]
noaatemp = pd.read_csv("https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/global/time-series/antarctic/land_ocean/12/11/1850-2023/data.csv", skiprows=5, names=noaanames, header=0)
noaatemp["temp"] = pd.to_numeric(noaatemp["temp"], errors="coerce")
noaatemp["temp"] = noaatemp["temp"] + (-54.5)
noaasort = noaatemp.sort_values(by="year", ascending=False)
noaasort.to_csv("noaatemp.csv", index=False)

##### Initital Analysis

In [ ]:
epicacols.head()

In [ ]:
epicacols.tail()

In [ ]:
epicacols.info()

In [ ]:
epicacols.describe()

In [ ]:
epicacols.columns

---

In [ ]:
noaasort.head()

In [ ]:
noaasort.tail()

In [ ]:
noaasort.info()

In [ ]:
noaasort.describe()

In [ ]:
noaasort.columns

---

---

### Rainfall

#### Rain Data 1 - Irish Rainfall

In [ ]:
raindfread = pd.read_csv("yearlyrain.csv")

In [ ]:
raindfread.head()

In [ ]:
raindfread.tail()

In [ ]:
raindfread.describe()

In [ ]:
raindfread.columns

#### Rain Data 2

For axtenstion of rainfall data and the later temperature data I have sourced the weather readings from our nearest neighbour, the UK, fromu their Met Historical Data. Specifically the Oxford weather station as it has been collecting data from 1853 and are still active today.

In [337]:
metengdata = "https://www.metoffice.gov.uk/pub/data/weather/uk/climate/stationdata/oxforddata.txt"




TypeError: read_csv() got an unexpected keyword argument 'error_bad_lines'

In [330]:
metengdfcols.head()

,yyyy mm tmax tmin af rain sun
1,1853 2 3.2 -1.8 19 29.3 ...
2,1853 3 7.7 -0.6 20 25.9 ...
3,1853 4 12.6 4.5 0 60.1 ...
4,1853 5 16.8 6.1 0 59.5 ...
5,1853 6 20.1 10.7 0 82.0 ...


In [331]:
metengdfcols.columns

Index(['year', 'month', 'temp', 'tmin', 'af', 'rain', 'sun'], dtype='object')


---



### Initial Data Adjustments

#### Data Clean-up

In [ ]:
epica8kyr2["year"] = epica8kyr2["year"].astype(int) #change data within the year column to an integer
epica8kyr2 = epica8kyr2.sort_values(by="year", ascending=False)  #sort column by year
epica8kyr2.to_csv("epica8kyr2.csv", index=False) #save to csv

##### CO2 - Merge Data

In [ ]:
mergeddata = pd.merge(epica8kyr3, epica8kyr2, on=["year", "co2", "unc"], how="outer")
mergeddata = pd.merge(mergeddata, epica8kyr1, on=["year", "co2"], how="outer")

mergeddata.to_csv("mergedco2data.csv", index=False)
mergeddata.columns #check columns merged correctly

#### CH4

##### CH4 - Merge Data

In [ ]:
mergech4 = pd.merge(ch4data, noaach4sort, on=["year", "ch4"], how="outer")
mergech4.to_csv("mergedch4.csv", index=False)

In [ ]:
mergech4.columns

#### Temperature

##### Temperature - Merge Data

In [ ]:
mergetemp = pd.merge(noaasort, epicacols, on=["year", "temp"], how="outer")
mergetemp.to_csv("tempsmerge.csv", index=False)

In [ ]:
mergetemp.head()

In [ ]:
mergetemp.shape #check the merge went correctly

### Statistical Measurement

### CO2

In [ ]:
mean_value = mergeddata["co2"].mean()
print(f"CO2 Data Mean Value: {mean_value}")

median_value = mergeddata["co2"].median()
print(f"CO2 Data Median Value: {median_value}")

std_deviation = mergeddata["co2"].std()
print(f"CO2 Data Standard Deviation: {std_deviation}")

correlation = mergeddata["co2"].corr(mergeddata["year"])
print(f"CO2 Data Standard Deviation: {correlation}")

### CH4

In [ ]:
mean_value = mergech4["ch4"].mean()
print(f"CH4 Data Mean Value: {mean_value}")

median_value = mergech4["ch4"].median()
print(f"CH4 Data Median Value: {median_value}")

std_deviation = mergech4["ch4"].std()
print(f"CH4 Data Standard Deviation: {std_deviation}")

correlation = mergech4["ch4"].corr(mergech4["year"])
print(f"CH4 Data Standard Deviation: {correlation}")

### TEMPERATURE

In [ ]:
mean_value = mergetemp["temp"].mean()
print(f"Temp Data Mean Value: {mean_value}")

median_value = mergetemp["temp"].median()
print(f"Temp Data Median Value: {median_value}")

std_deviation = mergetemp["temp"].std()
print(f"Temp Data Standard Deviation: {std_deviation}")

correlation = mergetemp["temp"].corr(mergetemp["year"])
print(f"Temp Data Standard Deviation: {correlation}")

## Irish Context

In [ ]:
weatherstations = pd.read_csv("DATA Files/weatherstationdata.csv")

weatherstations.columns
#to find the oldest weather station in the country that is still open to get a larger set of data
weatherstations["open year "] = pd.to_datetime(weatherstations["open year "], errors='coerce')

weatherstationsoldest = weatherstations.sort_values(by="open year ")
weatherstationsoldest.head()

### Temperature

#### Temperature - Merge Data

### Rainfall

In [ ]:
irelandrain = pd.read_csv("DATA Files/mly108.csv") 
irelandrain.tail() #check final year entry
irelandrain.head()

irelandrain["rain"] = irelandrain["rain"].astype(float)
uniqueyears = irelandrain["year"].unique()

totalbyyear = []

for year in uniqueyears:
    
    specificyear = irelandrain[irelandrain["year"] == year]
    totalmedianyear = specificyear["rain"].sum()
    totalbyyear.append({"year": year, "rain": totalmedianyear})

raindfnew = pd.DataFrame(totalbyyear)

raindfnew.to_csv("yearlyrain.csv", index=False)

#### Rainfall - Merge Data



---

## Analysis



### Plot Overview

In [ ]:
years = mergeddata["year"]
co2values = mergeddata["co2"]
ch4values = mergech4["ch4"]
tempvalues = mergetemp["temp"]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

sns.lineplot(x=years, y=tempvalues, ax=ax1, label="Temp")
sns.lineplot(x=years, y=co2values, ax=ax2, label="CO2")
sns.lineplot(x=years, y=ch4values, ax=ax2, label="CH4")

fig.suptitle("CO2 and CH4 Over Years")
ax1.set_xlabel("Year")

ax1.set_ylabel("Temp")
ax2.set_ylabel("CO2")
ax2.set_ylabel("CH4")

ax1.legend()
ax2.legend()

plt.tight_layout()

plt.show()


### CO2 - Plots

In [ ]:
plt.figure(figsize=(18, 16))
plt.plot(mergeddata["year"], mergeddata["co2"], color="blue")
plt.xlabel("YEAR", fontsize=20)
plt.ylabel("CO2", fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.title("CO2 V Time", fontsize=20)
plt.tight_layout()
plt.savefig("lineplot1.png")
#### Plot
#SNS Lineplot
#style
sns.set_style("whitegrid")
sns.set_context("paper") 

In [ ]:
plt.figure(figsize=(16, 12))

#lineplot
sns.lineplot(data=mergeddata, x="year", y="co2", color="green")

#labels
plt.xlabel("Year", fontsize=14)
plt.ylabel("CO2 Levels", fontsize=14)
plt.title("CO2 Levels Over Time", fontsize=14)
plt.xticks(rotation=45, fontsize=20)
plt.yticks(fontsize=20)


plt.tight_layout()  #additional spacing
plt.savefig("snslineplotnew.png")

### CH4 - Plots

In [ ]:
#style
sns.set_style("whitegrid")
sns.set_context("paper")

#size
plt.figure(figsize=(16, 12))

#lineplot
sns.lineplot(data=mergech4, x="year", y="ch4", color="blue")

#labels
plt.xlabel("Year", fontsize=12)
plt.ylabel("CH4", fontsize=12)
plt.title("CH4 Levels", fontsize=14)
plt.xticks(rotation=45)

#ticks spacing
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)


plt.tight_layout()  #additional spacing
plt.savefig("ch4levels2.png")

In [ ]:
plt.figure(figsize=(16, 12))
#lineplot
sns.lineplot(data=mergech4, x="year", y="ch4", color="blue")

#labels
plt.xlabel("Year", fontsize=12)
plt.ylabel("CH4", fontsize=12)
plt.title("CH4 Levels", fontsize=14)
plt.xticks(rotation=45)

#ticks spacing
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)


plt.tight_layout()  #additional spacing
plt.savefig("ch4levels2.png")

In [ ]:
plt.figure(figsize=(16, 12))
#selection of data for demonstration

sns.set_style("whitegrid")
sns.set_context("notebook")

sns.lineplot(data=mergeddata, x="year", y="co2", label="CO2")
sns.lineplot(data=ch4data, x="year", y="ch4", label="Mean CH4")

plt.title("CO2 and Mean CH4 from 8kyr before present")
plt.xlabel("Year")
plt.ylabel("Value")

plt.legend()

plt.show()

### Temperature - Plots

In [ ]:
plt.figure(figsize=(16, 12))
#selection of data for demonstration

sns.set_style("whitegrid")
sns.set_context("notebook")

sns.lineplot(data=mergeddata, x="year", y="co2", label="CO2")

plt.title("CO2")
plt.xlabel("Year")
plt.ylabel("Value")

plt.legend()

plt.show()

In [ ]:
plt.figure(figsize=(16, 12))
#selection of data for demonstration

sns.set_style("whitegrid")
sns.set_context("notebook")

sns.lineplot(data=mergetemp, x="year", y="temp", label="Temp")

plt.title("Temperature")
plt.xlabel("Year")
plt.ylabel("Value")

plt.legend()

plt.show()

### Rainfall - Plots

In [ ]:
sns.set_context("paper", rc={"lines.linewidth": 0.5, "figure.figsize":(10, 22)})
plot = sns.lineplot(data=raindfread, x="year", y="rain")
plot.set_xlabel("year")
plot.set_ylabel("Rainfall (mm)")

In [ ]:
plt.figure(figsize=(16, 12))

sns.set_style("whitegrid")
sns.set_context("notebook")

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 8), sharex=True)

raindfreadfiltered = raindfread[(raindfread["year"] >= 1980) & (raindfread["year"] <= 2023)]
mergetempfiltered = mergetemp[(mergetemp["year"] >= 1980) & (mergetemp["year"] <= 2023)]
mergech4filtered = mergech4[(mergech4["year"] >= 1980) & (mergech4["year"] <= 2023)]
mergeco2filtered = mergeddata[(mergeddata["year"] >= 1980) & (mergeddata["year"] <= 2023)]

ax1.set_title("Rain")
ax1.set_xlabel("Year 1980 - 2023")
ax1.set_ylabel("Rain mm")
sns.lineplot(data=raindfreadfiltered, x="year", y="rain", ax=ax1, label="Rain mm", color="green", marker="o")

ax2.set_title("Temperature")
ax2.set_xlabel("Year 1980 - 2023")
ax2.set_ylabel("Celcius")
sns.lineplot(data=mergetempfiltered, x="year", y="temp", ax=ax2, label="Celcius", color="orange", marker="o")

ax3.set_title("CO2")
ax3.set_xlabel("Year 1980 - 2023")
ax3.set_ylabel("CO2 ppm")
sns.lineplot(data=mergeco2filtered, x="year", y="co2", ax=ax3, label="CO2 ppm", color="blue", marker="o")

ax4.set_title("CH4")
ax4.set_xlabel("Year 1980 - 2023")
ax4.set_ylabel("CH4 ppm")
sns.lineplot(data=mergech4filtered, x="year", y="ch4", ax=ax4, label="CH4 ppm", color="red", marker="o")
ax4.set_xlabel("")

plt.subplots_adjust(hspace=1)
plt.legend()

plt.show()

### Trends


### Relationships



---

<a id="03i">

## Predictions

</a>

### Synthetic Data


### Comparisons with Published Climate Models

In [ ]:
#SECTION 3 - PYTHON CELL



---

<a id="07i">

## REFERENCES

</a>


---